In [ ]:
import os
import sys
import json
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pyplot import imread

sys.path.append("..")
import scripts.create_overview as create_overview

In [ ]:
args = {
    "data_path": "/Volumes/Extreme_SSD/MegaDepth/scenes/",
    # "data_path": "../data/scenes/",
    "output": "../data/metrics_overview.json",
}

# convert to argparse.Namespace
args = type("Namespace", (object,), args)


create_overview.main(args)

metrics = json.load(open("../data/overview.json"))

scenes = list(metrics.keys())
print(f"Loaded {len(scenes)} scenes")

In [ ]:
scene = "0229"

In [ ]:
def plot_metric(metrics, scene):
    keys = [
        "n_reg_images", 
        "perc_reg_images", 
        "mean_reprojection_error", 
        "n_observations", 
        "mean_obs_per_reg_image", 
        "mean_track_length",
        "mean_overlap"
    ]
    
    labels = []
    for m in metrics[scene]:
        if "model_name" in m.keys():
            labels.append(m["model_name"])
        else:
            labels.append(f"{m['features']}-{m['matcher']}")

    fig, ax = plt.subplots(4, 2, figsize=(30, 30))
    colors = ["red", "green", "blue", "orange", "purple", "brown", "pink", "gray", "olive", "cyan"]
    
    ids = np.argsort(labels)
    labels = [labels[i] for i in ids]
    
    for i, metric in enumerate(keys):
        values = [m[metric] for m in metrics[scene]]

        values = [values[i] for i in ids]

        ax[i // 2, i % 2].bar(labels, values)

        
        # rotate labels
        for tick in ax[i // 2, i % 2].get_xticklabels():
            tick.set_rotation(10)
        
        ax[i // 2, i % 2].set_title(metric)
        
    
    plt.suptitle(f"Scene {scene}-sparse")
    plt.show()

In [ ]:
plot_metric(metrics, scene)

In [ ]:
# print table

# model name & n_images & reg images & mean reprojection error & n observations & mean obs per reg image & mean track length & mean overlap

print("model name & n_images & reg images & mean reprojection error & n observations & mean obs per reg image & mean track length & mean overlap")
for m in metrics[scene]:
    if "model_name" in m.keys():
        model_name = m["model_name"]
    else:
        model_name = f"{m['features']}-{m['matcher']}"
    
    print(f"{model_name} & {m['n_images']} & {m['n_reg_images']} & {m['mean_reprojection_error']:.2f} & {m['n_observations']} & {m['mean_obs_per_reg_image']:.2f} & {m['mean_track_length']:.2f} & {m['mean_overlap']:.2f} \\\\")

In [ ]:
n_images = metrics[scene][0]["n_images"]

base_reg_images = next(m for m in metrics[scene] if m["model_name"] == "baseline")["n_reg_images"]
base_perc_reg_images = next(m for m in metrics[scene] if m["model_name"] == "baseline")["perc_reg_images"]

try:
    super_reg_images = next(m for m in metrics[scene] if m["model_name"] == "superpoint_max-superglue-netvlad-50")["n_reg_images"]
    super_perc_reg_images = next(m for m in metrics[scene] if m["model_name"] == "superpoint_max-superglue-netvlad-50")["perc_reg_images"]
    super_name = "superpoint_max-superglue-netvlad-50"
except Exception as e:
    try:
        super_reg_images = next(m for m in metrics[scene] if m["model_name"] == "superpoint_max-superglue-exhaustive-50")["n_reg_images"]
        super_perc_reg_images = next(m for m in metrics[scene] if m["model_name"] == "superpoint_max-superglue-exhaustive-50")["perc_reg_images"]
        super_name = "superpoint_max-superglue-exhaustive-50"
    except Exception as e:
        super_reg_images = next(m for m in metrics[scene] if m["model_name"] == "superpoint_max-superglue-exhaustive")["n_reg_images"]
        super_perc_reg_images = next(m for m in metrics[scene] if m["model_name"] == "superpoint_max-superglue-exhaustive")["perc_reg_images"]
        super_name = "superpoint_max-superglue-exhaustive"
except:
    raise ValueError("Could not find superpoint model")


print(f"Num images: {n_images}")
print(f"Baseline:   {base_reg_images} ({base_perc_reg_images:.2f}%)")
print(f"Superpoint: {super_reg_images} ({super_perc_reg_images:2f}%)")

In [ ]:
base_obs = next(m for m in metrics[scene] if m["model_name"] == "baseline")["n_observations"]
super_obs = next(m for m in metrics[scene] if m["model_name"] == super_name)["n_observations"]

print("Num observations")
print(f"Baseline:         {base_obs}")
print(f"Superpoint:       {super_obs}")

In [ ]:
base_mean_obs = next(m for m in metrics[scene] if m["model_name"] == "baseline")["mean_obs_per_reg_image"]
super_mean_obs = next(m for m in metrics[scene] if m["model_name"] == super_name)["mean_obs_per_reg_image"]

print("Mean observations per registered image")
print(f"Baseline:         {base_mean_obs}")
print(f"Superpoint:       {super_mean_obs}")

# Visualize Results

In [ ]:
from pathlib import Path

import pycolmap
from hloc import visualization
from hloc.utils import viz_3d

data_path = Path("/Volumes/Extreme_SSD/MegaDepth/scenes/")
# data_path = Path("../data")

In [ ]:
model_path = Path(os.path.join(data_path, scene, "sparse", super_name))
img_path = Path(os.path.join(data_path, scene, "images"))

super_model = pycolmap.Reconstruction(model_path)

super_track_lengths = [p.track.length() for p in super_model.points3D.values()]
super_n_visible = [img.num_points3D() for img in super_model.images.values()]

visualization.visualize_sfm_2d(super_model, img_path, color_by='visibility', n=3)

In [ ]:
visualization.visualize_sfm_2d(super_model, img_path, color_by='depth', n=3)

In [ ]:
visualization.visualize_sfm_2d(super_model, img_path, color_by='track_length', n=3)

In [ ]:
model_path = Path(os.path.join(data_path, scene, "sparse", "baseline"))

base_model = pycolmap.Reconstruction(model_path)

base_track_lengths = [p.track.length() for p in base_model.points3D.values()]
base_n_visible = [img.num_points3D() for img in base_model.images.values()]

visualization.visualize_sfm_2d(base_model, img_path, color_by='visibility', n=3)

In [ ]:
visualization.visualize_sfm_2d(base_model, img_path, color_by='depth', n=3)

In [ ]:
visualization.visualize_sfm_2d(base_model, img_path, color_by='track_length', n=3)

In [ ]:
# plot num observations per image

plt.hist(super_n_visible, bins=100, alpha=0.5, label="superpoint")
plt.hist(base_n_visible, bins=100, alpha=0.5, label="baseline")

plt.title("Num observations per image")
plt.legend()
plt.show()


In [ ]:
# plot track lengths

plt.hist(super_track_lengths, bins=100, alpha=0.5, label="superpoint", log=True)
plt.hist(base_track_lengths, bins=100, alpha=0.5, label="baseline", log=True)


plt.title("Track lengths")
plt.xlabel("Track length")
plt.ylabel("Frequency (log scale)")
plt.legend()
plt.show()

# Show Unregistered Images

In [ ]:
super_path = Path(os.path.join(data_path, scene, "sparse", super_name))
base_path = Path(os.path.join(data_path, scene, "sparse", "baseline"))

super_model = pycolmap.Reconstruction(super_path)
base_model = pycolmap.Reconstruction(base_path)

In [ ]:
images = os.listdir(img_path)
print(f"Scene {scene} has {len(images)} images")

In [ ]:
images_supermodel = [i.name for i in super_model.images.values()]
images_basemodel = [i.name for i in base_model.images.values()]

print(f"Superpoint model has {len(images_supermodel)} images")
print(f"Baseline model has {len(images_basemodel)} images")

In [ ]:
# get 25 random images that are not in the baseline model
unregistered_images = np.random.choice([i for i in images if i not in images_basemodel], 25)

fig, ax = plt.subplots(5, 5, figsize=(30, 30))
for i, image in enumerate(unregistered_images):
    ax[i // 5, i % 5].imshow(plt.imread(os.path.join(img_path, image)))
    in_supermodel = image in images_supermodel
    ax[i // 5, i % 5].set_title(f"{image} (in supermodel: {in_supermodel})")
    ax[i // 5, i % 5].axis("off")

plt.show()

In [ ]:
# get 25 random images that are not in the superpoint model
unregistered_images = np.random.choice([i for i in images if i not in images_supermodel], 25)

fig, ax = plt.subplots(5, 5, figsize=(30, 30))
for i, image in enumerate(unregistered_images):
    ax[i // 5, i % 5].imshow(plt.imread(os.path.join(img_path, image)))
    in_basemodel = image in images_basemodel
    ax[i // 5, i % 5].set_title(f"{image} (in basemodel: {in_basemodel})")
    ax[i // 5, i % 5].axis("off")

plt.show()

In [ ]:
from pathlib import Path
import h5py

import numpy as np
import pandas as pd

import torch
import collections.abc as collections


from hloc.utils.io import list_h5_names
from hloc.utils.read_write_model import read_images_binary
from hloc.utils.read_write_model import read_images_binary
from hloc.utils.io import list_h5_names
from hloc.utils.parsers import parse_image_lists

db_descriptors = None
# descriptors = Path("../data/0229/features/netvlad.h5")
descriptors = Path(os.path.join(data_path, scene, "features", "netvlad.h5"))
num_matched = 100
# output = Path("../data/retrieval.txt")
output = None
query_prefix=None
query_list=None
db_prefix=None
db_list=None
db_model=None
db_descriptors=None

image_dir = os.path.join(data_path, scene, "images")

def parse_names(prefix, names, names_all):
    if prefix is not None:
        if not isinstance(prefix, str):
            prefix = tuple(prefix)
        names = [n for n in names_all if n.startswith(prefix)]
        if len(names) == 0:
            raise ValueError(
                f'Could not find any image with the prefix `{prefix}`.')
    elif names is not None:
        if isinstance(names, (str, Path)):
            names = parse_image_lists(names)
        elif isinstance(names, collections.Iterable):
            names = list(names)
        else:
            raise ValueError(f'Unknown type of image list: {names}.'
                             'Provide either a list or a path to a list file.')
    else:
        names = names_all
    return names

def get_descriptors(names, path, name2idx=None, key='global_descriptor'):
    if name2idx is None:
        with h5py.File(str(path), 'r', libver='latest') as fd:
            desc = [fd[n][key].__array__() for n in names]
    else:
        desc = []
        for n in names:
            with h5py.File(str(path[name2idx[n]]), 'r', libver='latest') as fd:
                desc.append(fd[n][key].__array__())
    return torch.from_numpy(np.stack(desc, 0)).float()

if db_descriptors is None:
    db_descriptors = descriptors
if isinstance(db_descriptors, (Path, str)):
    db_descriptors = [db_descriptors]
name2db = {n: i for i, p in enumerate(db_descriptors)
            for n in list_h5_names(p)}
db_names_h5 = list(name2db.keys())
query_names_h5 = list_h5_names(descriptors)

if db_model:
    images = read_images_binary(db_model / 'images.bin')
    db_names = [i.name for i in images.values()]
else:
    db_names = parse_names(db_prefix, db_list, db_names_h5)
if len(db_names) == 0:
    raise ValueError('Could not find any database image.')
query_names = parse_names(query_prefix, query_list, query_names_h5)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
db_desc = get_descriptors(db_names, db_descriptors, name2db)
query_desc = get_descriptors(query_names, descriptors)
sim = torch.einsum('id,jd->ij', query_desc.to(device), db_desc.to(device))

# Avoid self-matching
self = np.array(query_names)[:, None] == np.array(db_names)[None]

In [ ]:
scores = torch.sum(sim > 0.2, dim=1) / sim.shape[1]

In [ ]:
img_names = [
    img for img in os.listdir(os.path.join(data_path, scene, "images")) 
    if img.endswith(".jpg") or img.endswith(".JPG") or img.endswith(".png")
]

r_img_names = []
for key in super_model.images.values():
    r_img_names.append(key.name)

df = pd.DataFrame({'query': query_names, 'score': scores, "registered": False})
df.set_index('query', inplace=True)
df.registered = df.index.isin(r_img_names)
df = df.sort_values(by=['registered', 'score'], ascending=[True, True])


fig, ax = plt.subplots(figsize=(10, 10))
ax.bar(df.index, df.score, color=df.registered.map({True: 'green', False: 'red'}))

ax.set_xticks([])

plt.title("Rel. Number of Scores Above 0.2 in Retrieval Matrix")

plt.ylabel("Rel. Number of Scores Above 0.2")
plt.xlabel("Query Image")

plt.show()

In [ ]:
# plot top 25 unreigstered images

cdf = df[~df.registered][:25]

# revert order
cdf = cdf.iloc[::-1]

fig, ax = plt.subplots(5, 5, figsize=(10, 10))
for i, (name, row) in enumerate(cdf.iterrows()):
    ax[i // 5, i % 5].imshow(imread(os.path.join(image_dir, name)))
    ax[i // 5, i % 5].set_title(f"{100*row.score:.1f} %")
    ax[i // 5, i % 5].set_xticks([])
    ax[i // 5, i % 5].set_yticks([])

fig.suptitle("Unreigstered images with lowest scores")
plt.show()

In [ ]:
cdf = df[~df.registered][-25:]

# revert order
cdf = cdf.iloc[::-1]

fig, ax = plt.subplots(5, 5, figsize=(10, 10))
for i, (name, row) in enumerate(cdf.iterrows()):
    ax[i // 5, i % 5].imshow(imread(os.path.join(image_dir, name)))
    ax[i // 5, i % 5].set_title(f"{100*row.score:.1f} %")
    ax[i // 5, i % 5].set_xticks([])
    ax[i // 5, i % 5].set_yticks([])

fig.suptitle("Unreigstered images with highest scores")
plt.show()

In [ ]:
cdf = df[df.registered][-25:]

# revert order
cdf = cdf.iloc[::-1]

fig, ax = plt.subplots(5, 5, figsize=(10, 10))
for i, (name, row) in enumerate(cdf.iterrows()):
    ax[i // 5, i % 5].imshow(imread(os.path.join(image_dir, name)))
    ax[i // 5, i % 5].set_title(f"{100*row.score:.1f} %")
    ax[i // 5, i % 5].set_xticks([])
    ax[i // 5, i % 5].set_yticks([])


fig.suptitle("Reigstered images with highest scores")
plt.show()

In [ ]:
cdf = df[df.registered][:25]

# revert order
cdf = cdf.iloc[::-1]

fig, ax = plt.subplots(5, 5, figsize=(10, 10))
for i, (name, row) in enumerate(cdf.iterrows()):
    ax[i // 5, i % 5].imshow(imread(os.path.join(image_dir, name)))
    ax[i // 5, i % 5].set_title(f"{100*row.score:.1f} %")
    ax[i // 5, i % 5].set_xticks([])
    ax[i // 5, i % 5].set_yticks([])

fig.suptitle("Reigstered images with lowest scores")
plt.show()

# Visualize Matches

In [ ]:
matches_path = os.path.join(data_path, scene, "matches", "superpoint_max-superglue-netvlad-50.h5")
# matches_path = os.path.join(data_path, scene, "matches", "superpoint_max-superglue-exhaustive.h5")
matches = h5py.File(matches_path, 'r')

In [ ]:
reg_images = [img.name for img in super_model.images.values()]
unreg_images = [img for img in img_names if img not in reg_images]

print(f"Number of registered images:   {len(reg_images)}")
print(f"Number of unregistered images: {len(unreg_images)}")

In [ ]:
# plot reg reg
from tqdm import tqdm

key_to_idx_reg = {key: idx for idx, key in enumerate(reg_images)}
key_to_idx_unreg = {key: idx for idx, key in enumerate(unreg_images)}

counts_reg_reg = np.zeros((len(reg_images), len(reg_images)))
for img1 in tqdm(reg_images):
    if img1 in matches.keys():
        for img2 in reg_images:
            if img2 in matches[img1].keys():
                counts_reg_reg[key_to_idx_reg[img1], key_to_idx_reg[img2]] = matches[img1][img2]["matches0"].shape[0]

count_reg_unreg = np.zeros((len(reg_images), len(unreg_images)))
for img1 in tqdm(reg_images):
    if img1 in matches.keys():
        for img2 in unreg_images:
            if img2 in matches[img1].keys():
                count_reg_unreg[key_to_idx_reg[img1], key_to_idx_unreg[img2]] = matches[img1][img2]["matches0"].shape[0]

counts_unreg_unreg = np.zeros((len(unreg_images), len(unreg_images)))
for img1 in tqdm(unreg_images):
    if img1 in matches.keys():
        for img2 in unreg_images:
            if img2 in matches[img1].keys():
                counts_unreg_unreg[key_to_idx_unreg[img1], key_to_idx_unreg[img2]] = matches[img1][img2]["matches0"].shape[0]

In [ ]:
max_count = np.max([np.sum(counts_reg_reg==0), np.sum(counts_unreg_unreg==0), np.sum(count_reg_unreg==0)])
max_count

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(30, 20))

size = 500

rel_zeros = np.sum(counts_reg_reg==0) / counts_reg_reg.size
ax[0, 0].imshow(np.log(counts_reg_reg+1)[:size, :size], cmap="gray")
ax[0, 0].set_title(f"Registered - Registered ({rel_zeros*100:.2f} % zeros)")
ax[0, 0].set_ylabel("Registered images")
ax[0, 0].set_xlabel("Registered images")

ax[1, 0].hist(counts_reg_reg.flatten(), bins=100)
ax[1, 0].set_yscale("log")
ax[1, 0].set_title("Registered - Registered")
ax[1, 0].set_ylabel("log(count)")
ax[1, 0].set_ylim(1, max_count)


rel_zeros = np.sum(count_reg_unreg==0) / count_reg_unreg.size
ax[0, 1].imshow(np.log(count_reg_unreg+1)[:size, :size], cmap="gray")
ax[0, 1].set_title(f"Registered - Unregistered ({rel_zeros*100:.2f} % zeros)")
ax[0, 1].set_ylabel("Registered images")
ax[0, 1].set_xlabel("Unregistered images")

ax[1, 1].hist(count_reg_unreg.flatten(), bins=100)
ax[1, 1].set_yscale("log")
ax[1, 1].set_title("Registered - Unregistered")
ax[1, 1].set_ylabel("log(count)")
ax[1, 1].set_ylim(1, max_count)


rel_zeros = np.sum(counts_unreg_unreg==0) / counts_unreg_unreg.size
ax[0, 2].imshow(np.log(counts_unreg_unreg+1)[:size, :size], cmap="gray")
ax[0, 2].set_title(f"Unregistered - Unregistered ({rel_zeros*100:.2f} % zeros)")
ax[0, 2].set_ylabel("Unregistered images")
ax[0, 2].set_xlabel("Unregistered images")

ax[1, 2].hist(counts_unreg_unreg.flatten(), bins=100)
ax[1, 2].set_yscale("log")
ax[1, 2].set_title("Unregistered - Unregistered")
ax[1, 2].set_ylabel("log(count)")
ax[1, 2].set_ylim(1, max_count)

plt.show()

# Overlap

In [ ]:
base_overlap_name = [m["overlap_fn"] for m in metrics[scene] if m["model_name"] == "baseline"][0]
super_overlap_name = [m["overlap_fn"] for m in metrics[scene] if m["model_name"] == super_name][0]

In [ ]:
base_sparse_overlap_matrix = np.load(os.path.join(data_path, scene, "metrics", "baseline", base_overlap_name))
super_sparse_overlap_matrix = np.load(os.path.join(data_path, scene, "metrics", super_name, super_overlap_name))

In [ ]:
plt.imshow(base_sparse_overlap_matrix)
plt.title("Baseline overlap matrix")
plt.show()

In [ ]:
plt.imshow(super_sparse_overlap_matrix)
plt.title("Superpoint overlap matrix")
plt.show()

In [ ]:
plt.hist(super_sparse_overlap_matrix.flatten(), bins=100, alpha=0.5, label="superpoint")
plt.hist(base_sparse_overlap_matrix.flatten(), bins=100, alpha=0.5, label="baseline")

plt.ylim(0, 1_000_000)

plt.title("Overlap histogram")
plt.legend()

plt.show()